In [1]:
import pandas as pd
df = pd.DataFrame()
import csv

## Extract data from raw text wifi log file

In [2]:
#Create csv file
jan1_csv = open('jan1.csv', 'w')
#Column names
col_name = ['building', 'date', 'month', 'time', 'device_count']
csv_writer = csv.DictWriter(jan1_csv, fieldnames=col_name)
csv_writer.writeheader()


39

In [3]:
def processLine(buildingName='UREC', wifi_log=''):
    count = 0
    for log_line in wifi_log:
        if buildingName in log_line:
            # process device count
            if '501080' in log_line or '501105' in log_line or '501106' in log_line:
                count += 0
            if '501093' in log_line:
                count += 1
            # Slice the line to get the date and time
            atr = log_line.split(' ')
            # Extract the data
            month = atr[0]
            date = atr[1]
            time = atr[2]
            building = buildingName
            # add data to csv file
            csv_writer.writerow({'building': building, 'date': date, 'month': month, 'time': time, 'device_count': count})


In [4]:
wifi_log = open('wireless_09-20-2021.log', 'r')
# # print first ten lines in the file
# for i in range(3):
#     print(wifi_log.readline())

# loop through each line in the file
processLine(wifi_log=wifi_log)

# close the file
wifi_log.close()


# Preprocess the raw csv to something meaningful (non-scalable)

### Process the time columns into a timestamp column

In [13]:
# convert jan1.csv to dataframe
df = pd.read_csv('jan1.csv')
# convert time to datetime
df['timestamp'] = pd.to_datetime(df['time'])
# get type of time column
print('timestamp column type: ', df['timestamp'].dtype)
# diplay from time 00:00:00 to 00:00:59
df1 = df.loc[(df['timestamp'] >= '17:00:00') & (df['timestamp'] <= '18:00:00')]
# drop date and month column
df1 = df1.drop(['date', 'month', 'time'], axis=1)
df1


timestamp column type:  datetime64[ns]


,building,device_count,timestamp
607155,UREC,99701,2022-11-04 17:00:00
607156,UREC,99701,2022-11-04 17:00:00
607157,UREC,99701,2022-11-04 17:00:00
607158,UREC,99701,2022-11-04 17:00:00
607159,UREC,99701,2022-11-04 17:00:00
...,...,...,...
689313,UREC,113123,2022-11-04 18:00:00
689314,UREC,113124,2022-11-04 18:00:00
689315,UREC,113124,2022-11-04 18:00:00
689316,UREC,113124,2022-11-04 18:00:00
